In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests
!pip install --upgrade xlrd #colab bug verson xlrd
!pip install geopandas
!pip install pandas fiona shapely pyproj rtree
!pip install contextily
!pip install pygeos
!pip install topojson
import geopandas as gpd
import contextily as ctx
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
#!pip install requests py7zr geopandas openpyxl tqdm s3fs PyYAML xlrd
#!pip install git+https://github.com/inseefrlab/cartiflette@80b8a5a28371feb6df31d55bcc2617948a5f9b1a

In [ ]:
#test_1 = pd.read_csv("https://www.data.gouv.fr/fr/datasets/r/5f71ba43-afc8-43a0-b306-dafe29940f9c", sep = ";", encoding = 'utf-8', engine = 'python')
#le lien du dossier data.gouv : 
# https://www.data.gouv.fr/fr/datasets/effectif-de-patients-par-pathologie-sexe-classe-dage-et-territoire-departement-region/
# methode "classique" pour obtenir un dataset a partir d un lien 
# probleme d encodage d ou les parametres sep, encoding, engine
# A FAIRE : COPIER LA BASE DE FACON LOCALE ET S Y REFERER PAR LA SUITE PAR EFFECTIFS.CSV 

In [ ]:
#Importation de la base détaillant les origines de la pollution
data_pollution = pd.read_csv("https://koumoul.com/data-fair/api/v1/datasets/igt-pouvoir-de-rechauffement-global/convert", encoding = 'utf-8')

#Dictionnaire associant les numéros de département à ceux des régions

correspondance_reg = {
    84: ['01', '03', '07', '15', '26', '38', '42', '43', '63', '69', '73', '74'],  # Auvergne-Rhône-Alpes
    27: ['21', '25', '39', '58', '70', '71', '89', '90'],  # Bourgogne-Franche-Comté
    53: ['22', '29', '35', '56'],  # Bretagne
    28: ['18', '28', '36', '37', '41', '45'],  # Centre-Val de Loire
    94: ['2A', '2B'],  # Corse
    44: ['08', '10', '51', '52', '54', '55', '57', '67', '68', '88'],  # Grand Est
    32: ['02', '59', '60', '62', '80'],  # Hauts-de-France
    11: ['75', '77', '78', '91', '92', '93', '94', '95'],  # Île-de-France
    76: ['14', '27', '50', '61', '76'],  # Normandie
    75: ['16', '17', '19', '23', '24', '33', '40', '47', '64', '79', '86', '87'],  # Nouvelle-Aquitaine
    24: ['09', '11', '12', '30', '31', '32', '34', '46', '48', '65', '66', '81', '82'],  # Occitanie
    52: ['44', '49', '53', '72', '85'],  # Pays de la Loire
    93: ['04', '05', '06', '13', '83', '84']  # Provence-Alpes-Côte d'Azur
}

correspondance_reg = {dep: region for region, deps in correspondance_reg.items() for dep in deps}

#Création de la variable département à partir du code commune de l'INSEE
data_pollution['DEP'] = data_pollution['INSEE commune'].str[:2]

#Chose à laquelle il faut faire attention : 
#DEP est de type serie car il ne contient pas que de variables numériques (2A et 2B)

#Création de la variable région à partir de celle du département
size = len(data_pollution['DEP'])
data_pollution['REG'] = pd.Series([0]*size)

for i in range(size) :
    data_pollution.loc[i, 'REG'] = correspondance_reg.get(data_pollution['DEP'][i],"inconnu")

REG_DEP_na = data_pollution[(data_pollution['DEP'].isna()) | (data_pollution['REG'].isna())]
REG_DEP_na.shape
#L'avantage par rapport au code précédent est qu'il n'y a plus de données perdues 
#ce qui n'était pas le cas avec le merge avec base_geo_pour_merge 

#Somme des niveaux de pollution par département
pollution_group_dep = data_pollution.drop(['Commune','INSEE commune', 'REG'], axis = 1).groupby('DEP', as_index=False).sum()
pollution_group_dep

#Somme des niveaux de pollution par région
pollution_group_reg = data_pollution.drop(['Commune','INSEE commune', 'DEP'], axis = 1).groupby('REG', as_index = False).sum()


#Dictionnaire de correspondance des numéro de région avec leur nom
#Utile pour l'affiche des graphiques
nom_reg = {
    84: 'Auvergne-Rhône-Alpes',
    27: 'Bourgogne-Franche-Comté',
    53: 'Bretagne',
    28: 'Centre-Val de Loire',
    94: 'Corse',
    44: 'Grand Est',
    32: 'Hauts-de-France',
    11: 'Île-de-France',
    76: 'Normandie',
    75: 'Nouvelle-Aquitaine',
    24: 'Occitanie',
    52: 'Pays de la Loire',
    93: "Provence-Alpes-Côte d'Azur"
}

# Conversion de la colonne 'REG' en type int
pollution_group_reg['REG'] = pollution_group_reg['REG'].astype(int)

# Création de la variable 'Nom de la région' à partir de celle de la région
pollution_group_reg['Nom de la région'] = pollution_group_reg['REG'].map(nom_reg).fillna("inconnu")

# Réaranger l'ordre des colonnes
colonnes_ordre = ['REG', 'Nom de la région'] + [colonne for colonne in pollution_group_reg.columns if colonne not in ['REG', 'Nom de la région']]
pollution_group_reg = pollution_group_reg[colonnes_ordre]

set_dep_littoral = (59, 62, 80, 76, 27, 14, 50, 35, 
                      22, 29, 56, 44, 85, 17, 33, 40, 
                    64, 66, 11, 34,
                    30, 13, 83, 0o6, '2A', '2B')
# Importation des bases en local pour faire tourner le code main
data_pollution.to_csv('base_pollution.csv', sep=',', index=False, encoding='utf-8')
pollution_group_reg.to_csv('pollution_group_reg.csv', sep=',', index=False, encoding='utf-8')
pollution_group_dep.to_csv('pollution_group_dep.csv', sep=',', index=False, encoding='utf-8')

In [ ]:
url = 'https://www.insee.fr/fr/statistiques/fichier/1893198/estim-pop-dep-sexe-gca-1975-2023.xls'

noms_colonnes=['numero_dep','dep','total_habitants']
pop_test = pd.read_excel(url, sheet_name='2023',skiprows=3,header=1, usecols = [0,1,7], skipfooter=4, names=noms_colonnes)


#pop_test2 = pop_test.set_index(['Num_dep ', 'Nom_dep'])
#pop_test2

#pop_test3 = pop_test.drop(df.index[:3])
#print(pop_test3)

pop_test.to_csv('base_pop_dep.csv')

In [ ]:
test_2 = pd.read_csv("effectifs.csv", sep = ";", encoding = 'utf-8', engine = 'python')

#commande a utiliser une fois qu on a la base 
#copiee en local 
#la base au prealable telechargee sur ordi perso
#Pour copier en local, glisser à gauche la table depuis les fichiers de l'ordi*

In [ ]:
groupe_dep = pd.read_csv("pollution_group_dep.csv", encoding = 'utf-8')

In [ ]:
test_2.head()
test_2.shape

In [ ]:
print("nombre de categories differentes pour niv2 est :"+str(len(test_2['patho_niv2'].unique())))
print("nombre de categories differentes pour niv3 est :"+str(len(test_2['patho_niv3'].unique())))

In [ ]:
#potentiellement creer 123 colonnes pour pouvoir faire du clustering 
#comparer les clusters obtenus par les caracteristiques d un inidivu pour voir si ca matche ou pas 
print(test_2['patho_niv1'].unique())

In [ ]:
data_gen = test_2.dropna()

In [ ]:
print(test_2.shape, data_gen.shape)

In [ ]:
liste_diff_patho1 = data_gen['patho_niv1'].unique()
liste_diff_patho2 = data_gen['patho_niv2'].unique()
liste_diff_patho3 = data_gen['patho_niv3'].unique()

print(liste_diff_patho1, "\n", liste_diff_patho2, "\n", liste_diff_patho3)

In [ ]:
print(data_gen.shape)

In [ ]:
#Creation d une colonne individus
data_gen.reset_index(inplace = True, names='ind') 



In [ ]:
data_gen.shape

In [ ]:
#tentative de compréhension de ce que la variable npop représente
#b = data_gen['npop'].unique()
#b = pd.DataFrame(b)

In [ ]:
#b = b.dropna()

In [ ]:
#b = b.set_axis(['test'], axis = 'columns')

In [ ]:
#b.head()

In [ ]:
#sum(b['test'])

In [ ]:
#resultat peu concluant

In [ ]:
#creation d indicatrices pour futur clustering
#aprs correction, utilisation de la fonction get dummies

In [ ]:
#liste_diff_patho3 = list(liste_diff_patho3)

In [ ]:
#premier code pour creation d indicatrice

In [ ]:
#liste_var = data_gen.columns[17:]
#liste_bis = []
#for i in liste_var : 
#    liste_bis.append(i)


In [ ]:
#indicatrice_patho_niv1 = liste_bis

In [ ]:
liste_auxil = data_gen['cla_age_5'].unique()
print(liste_auxil)

#afin de pouvoir clusteriser par la suite, nous allons creer des classes d age qui sont "discretes"
#nous faisons ce choix car les classes d age sont relativement homogene et en prenant la moyenne des deux 
#bornes pour les classes, nous gardons l idee d une distance d age qui pourrait etre utile par la suite
"""
inter_excl = ('95et+', 'tsage', np.nan)

inter_def = []
for i in liste_auxil :
    if i in inter_excl : 
        inter_def = inter_def
    else : 
        inter_def.append(i)


classe_age = []
for i in data_gen['cla_age_5'] : 
    if i in inter_def :
        temp = i.split('-')
        temp[0] = int(temp[0])
        temp[1] = int(temp[1])
        age = (temp[1] + temp[0])/2
    elif i == 'tsage' : 
        age = np.nan
    elif i == '95et+' : 
        age = 100
    else : 
        age = np.nan
    classe_age.append(age)
"""
#pb a regler, il manque un truc pour eviter que ca bug fort
#probablement type a regler ou categorie a revoir
#code peu ergonomique mais solution la plus sure, 
#toutes les autres tentatives en posant des conditions sur le type de i 
#n ont pas ete solde par une reussite
#on a print nombre de valeurs de classe_age et le code fonctionne



data_gen["classe_age"] = data_gen["cla_age_5"].apply(
    lambda x: np.mean(list(map(lambda y : int(y), x.split("-")))) if "-" in x else (np.nan if x == 'tsage' else 100)
)

#fonction ecrite par Antoine Palazzolo

In [ ]:
#on cree un nouveau dataframe pour pouvoir faire des manipulations plus rapides sur le data_gen

In [ ]:
df_model = data_gen[['ind','classe_age' , 'region', 'dept']]

In [ ]:
#truc bizarre a reverifier mais il se peut qu'il y ait eu un petit bug avec une colonne nan_ind

In [ ]:
#l'autre base pour la pollution ne prend pas les départements d'outre-mer en compte
#il est donc probable que l'on soit contraints de les exclure de l'analyse 


In [ ]:
df_model_test = pd.merge(df_model, groupe_dep, how = 'left', left_on='dept', right_on='DEP')

In [ ]:
#decision prise apres cette ligne de reduire 
#considerablement la taille de la base sans quoi aucun calcul ne serait possible
#j'ai essaye d'augmenter drastiquement les ressources disponibles pour 
#pouvoir ne serait-ce que merge et pour le clustering, 
#le calcul semble impossible

In [ ]:
#data_sans_na = data_gen[data_gen['patho_niv1'].apply(lambda s: s != 'Pas de pathologies repérées, traitements, maternité, hospitalisations ni traitement antalgique ou anti-inflammatoire')]

In [ ]:
df_model_reduced = data_gen[['ind','classe_age' , 'region', 'dept']]

In [ ]:
#meme en ayant divise par deux la taille de la base,
#augmente de facon considerable les ressources disponibles,
#efface tous les outputs des cellules
#ca ne marche pas 
#donc il faudra faire des regroupements
#vient la question : lesquels

In [ ]:
data_gen.drop(['patho_niv1', 'patho_niv2', 'patho_niv3', 'top'], axis = 1).sort_values(by = 'dept', ascending=True)

In [ ]:
pd.get_dummies(data_gen, prefix = 'indicatrice_', columns=['patho_niv1'], dtype='int')

In [ ]:
pd.get_dummies(data_gen, prefix = 'indicatrice_', columns=['patho_niv1'], dtype='int').dropna().to_csv('data_dummies.csv')

In [ ]:
df_modelisation = pd.get_dummies(data_gen, prefix = 'indicatrice', columns=['patho_niv1'], dtype='int')
age_30_60 = [32, 37, 42, 47, 52, 57, 62]
df_modelisation_30_60 = df_modelisation[df_modelisation['classe_age'].isin(age_30_60)]
df_modelisation_30_60 = df_modelisation_30_60[['classe_age'] + list(df_modelisation_30_60.columns.difference(['classe_age']))]

In [ ]:
df_modelisation = df_modelisation.drop(['ind','patho_niv2', 'patho_niv3', 'top', 'cla_age_5', 'sexe', 'region', 'ntop', 'npop', 'prev', 'niveau_prioritaire', 'libelle_classe_age', 
                      'libelle_sexe', 'tri', 'annee'], axis = 1)

df_modelisation_30_60 = df_modelisation_30_60.drop(['ind','patho_niv2', 'patho_niv3', 'top', 'cla_age_5', 'sexe', 'region', 'ntop', 'npop', 'prev', 'niveau_prioritaire', 'libelle_classe_age', 
                      'libelle_sexe', 'tri', 'annee'], axis = 1)

In [ ]:
dico = {'classe_age' : 'mean',
       'indicatrice_Autres affections de longue durée (dont 31 et 32)' : 'sum',
       'indicatrice_Cancers' : 'sum', 'indicatrice_Diabète' : 'sum',
       'indicatrice_Hospitalisations ponctuelles (avec ou sans pathologies, traitements ou maternité)' : 'sum',
       'indicatrice_Insuffisance rénale chronique terminale' : 'sum',
       'indicatrice_Maladies cardio-neurovasculaires' : 'sum',
       'indicatrice_Maladies du foie ou du pancréas (hors mucoviscidose)' : 'sum',
       'indicatrice_Maladies inflammatoires ou rares ou VIH ou SIDA' : 'sum',
       'indicatrice_Maladies neurologiques ou dégénératives' : 'sum',
       'indicatrice_Maladies psychiatriques' : 'sum',
       'indicatrice_Maladies respiratoires chroniques (hors mucoviscidose)' : 'sum',
       'indicatrice_Maternité (avec ou sans pathologies)' : 'sum',
       'indicatrice_Séjours en hospitalisation complète pour prise en charge de la Covid-19' : 'sum',
       'indicatrice_Traitements du risque vasculaire (hors pathologies)' : 'sum',
       'indicatrice_Traitements psychotropes (hors pathologies)' : 'sum'}

In [ ]:
df_modelisation = df_modelisation.groupby('dept').agg(dico).reset_index()
df_modelisation.to_csv('base_groupe_dep_moy_age.csv')
df_modelisation_30_60 = df_modelisation_30_60.groupby('dept').agg(dico).reset_index()


In [ ]:
df_modelisation['plus_repandu'] = df_modelisation.drop(['dept', 'classe_age',
                                                        'indicatrice_Maladies cardio-neurovasculaires',
                                                        'indicatrice_Traitements psychotropes (hors pathologies)'], axis = 1).apply(lambda x: x.idxmax(), axis = 1)
df_modelisation_30_60['plus_repandu'] = df_modelisation_30_60.drop(['dept', 'classe_age',
                                                        'indicatrice_Maladies cardio-neurovasculaires',
                                                        'indicatrice_Traitements psychotropes (hors pathologies)'], axis = 1).apply(lambda x: x.idxmax(), axis = 1)


In [ ]:
df_modelisation['plus_repandu'].unique()
#on aura donc peut-etre a considerer les variables avc et soins psychotropes
#differemment car elles risquent de peut-etre
#capter toute l analyse et ne pas laisser grand chose aux autres pathologies
#car dans le cas ou on les laisse dans le dataframe
#ce sont les deux seules maladies a apparaitre

In [ ]:
base_pollution = pd.read_csv('base_pollution.csv')

In [ ]:
base_pollution = base_pollution.groupby('DEP').sum()

In [ ]:
base_pollution = base_pollution.drop(['INSEE commune', 'Commune', 'REG'], axis = 1)

In [ ]:
base_pol_dep = df_modelisation.merge(base_pollution, how = 'right', left_on = 'dept', right_on = 'DEP',
                                     right_index=True)

base_pol_dep_30_60 = df_modelisation_30_60.merge(base_pollution, how = 'right', left_on = 'dept', right_on = 'DEP',
                                     right_index=True)

In [ ]:
base_pol_dep_gen = base_pol_dep.merge(pd.read_csv('base_pop_dep.csv'), how = 'left', left_on = 'dept', right_on = 'numero_dep')
base_pol_dep_gen_30_60 = base_pol_dep_30_60.merge(pd.read_csv('base_pop_dep.csv'), how = 'left', left_on = 'dept', right_on = 'numero_dep')

In [ ]:
base_pol_dep_gen.columns

In [ ]:
base_pol_dep_gen = base_pol_dep_gen.drop(['Unnamed: 0', 'numero_dep'], axis = 1)
base_pol_dep_gen_30_60 = base_pol_dep_gen_30_60.drop(['Unnamed: 0', 'numero_dep'], axis = 1)

In [ ]:
num_cols = base_pol_dep_gen.select_dtypes(include=['number']).columns
num_data = base_pol_dep_gen[num_cols]

In [ ]:
num_cols_30_60 = base_pol_dep_gen_30_60.select_dtypes(include=['number']).columns
num_data_30_60 = base_pol_dep_gen_30_60[num_cols_30_60]

In [ ]:
scaler = StandardScaler()
data_standard = scaler.fit_transform(num_data)
data_standard_30_60 = scaler.fit_transform(num_data_30_60)

In [ ]:
base_standard = pd.concat([base_pol_dep_gen[['dept', 'dep']], pd.DataFrame(data_standard, columns=num_cols)], axis=1)
base_standard_30_60 = pd.concat([base_pol_dep_gen_30_60[['dept', 'dep']], pd.DataFrame(data_standard_30_60, columns=num_cols_30_60)]
                                , axis=1)




In [ ]:
#on perd indicatrice_Hospitalisations ponctuelles (avec ou sans pathologies, traitements ou maternité), 
#etrange car toutes les lignes valent 0 mais mis a part ca c est bien 

In [ ]:
base_standard.to_csv("base_modelisation.csv")
base_standard_30_60.dropna().to_csv("base_modelisation_30_60.csv")